# Projeto 2 - Ciência dos Dados

Nome: Henrique Mualem Marti

Nome: Victor Vergara Arvoverde Albuquerque Cavalcanti

Nome: Edgard Neto

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji

In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
import emoji
import functools
import operator
from time import sleep
from random import shuffle

___
## Autenticando no  Twitter

* Conta: @Maul_bot

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass.json') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'coca'

#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 250

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     msgs.append(msg.full_text.lower())
#     i += 1
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [ ]:
#Twitts Treino
file_tre = pd.read_excel('coca.xlsx' , sheet_name='Treinamento')
#Twitts Teste
file_tes = pd.read_excel('coca.xlsx' , sheet_name='Teste')
file_tre.head()

#### Legenda de classificação:
0: Muito relevante - os *twetts* que expressam opiniões claras e explícitas sobre a bebida "Coca-Cola".

1: Relevante - os *twetts* que expressam alguma opinião sobre a bebida "Coca-Cola".

2: Neutro - os *twetts* que não expressam nenhuma opinião sobre a bebida "Coca-Cola".

3: Irrelevante - os *twetts* que não estão relacionados a bebida "Coca-Cola".


In [ ]:
#Limpeza

def stclear(Column):

    Column= Column.replace(' " '," ").replace(" ' "," ").replace(' "'," ").replace('" '," ")\
    .replace('"'," ")
    return Column


def cleanup(Column):

    Column= Column.replace("ç","c").replace("â","a").replace("ã","a").replace("à","a")\
    .replace("ê","e").replace("é","e").replace("ô","o").replace("õ","o").replace("/"," ")\
    .replace("?"," ").replace("!"," ").replace("@"," ").replace("\n"," ").replace(";"," ")\
    .replace(",","").replace(":"," ").replace("["," ").replace("]"," ").replace("}"," ")\
    .replace("{"," ").replace("\\"," ").replace(' " '," ").replace(" ' ","").replace(" * "," ")\
    .replace(" _ "," ").replace(" - "," ").replace(" . "," ").replace(" rt "," ").replace("…","")\
    .replace("“","").replace("             "," ").replace("   ","").replace(".."," ")\
    .replace("..."," ").replace("...."," ").replace("....."," ").replace("     "," ")\
    .replace("\ ","").replace("*","").replace(")","")  
    return Column

#for items in file.Treinamento:
    #p=cleanup(items)
    

In [ ]:
def rtcleaner(text):
    n = 0
    out = []
    while n in range(len(text)):
        if text[n] == "r":
            if text[n+1] =="t":
                if text[n+2] ==" ":
                    if text[n+3] =="@":
                        x = n
                        while text[x] != ":":
                            x = x+1
                        n = x
                        
        if text[n] == "h":
            if text[n+1] =="t":
                if text[n+2] =="t":
                    if text[n+3] =="p":
                        x = n
                        while text[x] != " ":
                            x = x+1
                        n = x

        if text[n] == "@":
            x = n
            while text[x] != " ":
                if x == len(text)-1:
                    break
                else:
                    x = x+1
            n = x

        out.append(text[n])
        n+=1
    out = "".join(out)
    return out

In [ ]:
#separação de emoji do resto do texto
#Link do fórum nas referências
def emoji(emoj):
    
    separa_emoji = emoji.get_emoji_regexp().split(emoj)
    emoji_separado = [substr.split() for substr in separa_emoji]
    emoji_separado = functools.reduce(operator.concat, em_separado)
    return emoji_separado

In [ ]:
file_tre.Treinamento = file_tre.Treinamento.apply(lambda x: x.lower())
#file_tre.Treinamento = file.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
file_tre.Treinamento = file_tre.Treinamento.apply(lambda x: x.translate(str.maketrans('','',string.digits)))
file_tre['Treinamento']=file_tre['Treinamento'].apply(stclear)

file_tes['Teste']=file_tes['Teste'].apply(stclear)
file_tes['Teste']=file_tes['Teste'].apply(cleanup)

In [ ]:
file_tre

In [ ]:
# Transformando quantitativa em qualitativa
file_tre.Relevância = file_tre.Relevância.astype('category')
file_tre.Relevância.head()

In [ ]:
contagem_relevancia = file_tre.Relevância.value_counts(True)
contagem_relevancia

In [ ]:
prob_MR = contagem_relevancia[0]
prob_R = contagem_relevancia[1]
prob_N = contagem_relevancia[2]
prob_IR = contagem_relevancia[3]

In [ ]:
file_MR=file_tre.loc[file_tre.Relevância==0,:]
file_R=file_tre.loc[file_tre.Relevância==1,:]
file_N=file_tre.loc[file_tre.Relevância==2,:]
file_IR=file_tre.loc[file_tre.Relevância==3,:]
MR_txt= " ".join(file_MR.Treinamento)
R_txt= " ".join(file_R.Treinamento)
N_txt= " ".join(file_N.Treinamento)
IR_txt= " ".join(file_IR.Treinamento)

In [ ]:
N_txt = rtcleaner(N_txt)
MR_txt = rtcleaner(MR_txt)
R_txt = rtcleaner(R_txt)
IR_txt = rtcleaner(IR_txt)

In [ ]:
N_txt = cleanup(N_txt)
MR_txt = cleanup(MR_txt)
R_txt = cleanup(R_txt)
IR_txt = cleanup(IR_txt)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
serie_MR = pd.Series(MR_txt.split())
serie_R = pd.Series(R_txt.split())
serie_N = pd.Series(N_txt.split())
serie_IR = pd.Series(IR_txt.split())

In [ ]:
tabela_MR = serie_MR.value_counts()
tabela_R = serie_R.value_counts()
tabela_N = serie_N.value_counts()
tabela_IR = serie_IR.value_counts()

In [ ]:
tabela_relativa_MR = serie_MR.value_counts(True)
tabela_relativa_R = serie_R.value_counts(True)
tabela_relativa_N = serie_N.value_counts(True)
tabela_relativa_IR = serie_IR.value_counts(True)
tabela_relativa_IR['nao']

In [ ]:
item_MR = tabela_MR[1]
item_R = tabela_R[1]
item_N = tabela_N[1]
item_IR = tabela_IR[1]

In [ ]:
def probPalavra(palavra):
    if palavra in tabela_MR:
        a = tabela_MR[palavra]
    else:
        a = 0
    if palavra in tabela_R:
        b = tabela_R[palavra]
    else:
        b = 0
    if palavra in tabela_N:
        c = tabela_N[palavra]
    else:
        c = 0
    if palavra in tabela_IR:
        d = tabela_IR[palavra]
    else:
        d = 0
    probabilidade_MR = ((a+1)/(len(serie_MR)+item_MR))*prob_MR
    probabilidade_R = ((b+1)/(len(serie_R)+item_R))*prob_R
    probabilidade_N = ((c+1)/(len(serie_N)+item_N))*prob_N
    probabilidade_IR = ((d+1)/(len(serie_IR)+item_IR))*prob_IR
    
    return [probabilidade_MR,probabilidade_R,probabilidade_N,probabilidade_IR]
        
print(probPalavra("amo"))

In [ ]:
def probFrase(frase):
    f = str(frase.lower())
    f = cleanup(f)
    coef_MR = 1
    coef_R = 1
    coef_N = 1
    coef_IR = 1
    
    for e in f:
        coef_MR *= probPalavra(e)[0]
        coef_R *= probPalavra(e)[1]
        coef_N *= probPalavra(e)[2]
        coef_IR *= probPalavra(e)[3]
        
    if coef_MR>coef_R and coef_MR>coef_N and coef_MR>coef_IR:
        return 0
        
    elif coef_R>coef_MR and coef_R>coef_N and coef_R>coef_IR:
        return 1
            
    elif coef_N>coef_MR and coef_N>coef_R and coef_N>coef_IR:
        return 2
    
    else:
        return 3
        
    
    
    
    
#print(probFrase("eu quero uma coca gelada"))
print(probFrase("as queimadas da Amazônia são responsabilidades do Brasil"))

In [ ]:
sim = []
for e in file_tes['Teste']:
    sim.append(probFrase(e))
file_tes['Simulação'] = sim

In [ ]:
#file_tes['Simulação']

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
total = len(file_tes['Relevância'])
MR_certos = 0
MR_errados = 0
R_certos = 0
R_errados = 0
N_certos = 0
N_errados = 0
IR_certos = 0
IR_errados = 0

for [e,t] in zip(file_tes['Relevância'], file_tes['Simulação']):
    if e == 0:
        if t == 0:
            MR_certos += 1
        else:
            MR_errados += 1
    if e == 1:
        if t == 1:
            R_certos += 1
        else:
            R_errados += 1
    if e == 2:
        if t == 2:
            N_certos += 1
        else:
            N_errados += 1
    if e == 3:
        if t == 3:
            IR_certos += 1
        else:
            IR_errados +=1
#help(zip)

In [ ]:

print("Acertos")
print("----------------------")
print("Muito Relevantes: ", MR_certos)
print("Relevantes: ", R_certos)
print("Neutros: ", N_certos)
print("Irrelevantes: ", IR_certos)
print("----------------------")
print("Erros")
print("----------------------")
print("Muito Relevantes errados: ", MR_errados)
print("Relevantes errados: ", R_errados)
print("Neutros errados: ", N_errados)
print("Irrelevantes errados: ", IR_errados)
print("----------------------")

In [ ]:
porcentual_acertos = (((MR_certos/total)+(R_certos/total)+(N_certos/total)+(IR_certos/total)))*100
porcentual_errados = (((MR_errados/total)+(R_errados/total)+(N_errados/total)+(IR_errados/total)))*100
print("Porcentagem de acertos: ", porcentual_acertos, "%")
print("Porcentagem de erros: ", porcentual_errados, "%")

In [ ]:

n = 300
while True:
    break
    api = tweepy.API(auth)

    #Inicia a captura, para mais detalhes: ver a documentação do tweepy
    i = 1
    msgs = []
    for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
        # Evitando mensagens repitidas:
        if msg not in msgs: 
            msgs.append(msg.full_text.lower())
            i += 1
        if i > n:
            break
    df = pd.DataFrame({'Teste': pd.Series(msgs)})
    df['Teste'] = file_tes['Teste'].apply(cleanup)
    df.dropna(inplace=True)
    sim=[]
    for e in df['Teste']:
        sim.append(probFrase(e))
    df['Simulação'] = sim
    MR = 0
    R = 0
    N = 0
    IR = 0
    for e in file_tes['Relevância']:
        if e==0:
            MR+=1
        if e==1:
            R+=1
        if e==2:
            N+=1
        else:
            IR+=1
    print("Muito relevantes: ", round((MR/(MR+R+N+IR)),2))
    print("Relevantes: ", round((R/(MR+R+N+IR)),2))
    print("Neutros: ", round((N/(MR+R+N+IR)),2))
    print("Irrelevantes: ", round((IR/(MR+R+N+IR)),2))
    sleep(60*60)
 

___
### Concluindo

Após a análise é possível concluir que o classificador criado não é eficaz para a classificação, pois apresenta apenas 30% de acerto, assim não deveria ser utilizado para a classificação de twitts em níveis de relevância para o setor de marketing da "Coca-Cola". Caso fossem utilizados mais twitts para o treinamento, provavelmete o índice de acerto seria maior, pois assim teria um aumento na quantidade de palavras do sistema.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis (*feito*)
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B) (*feito com 4 categorias*)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[How to split emoji from each other python?](https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python) **Função utilizada**